# Environment Kurulumunda Kullanılanlar
1)  conda create -n qwen python=3.10 -y
2)  conda activate qwen
3)  pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
4)  pip install transformers pillow requests bitsandbytes accelerate ipywidgets tqdm
5)  jupyter nbextension enable --py widgetsnbextension
6)  pip install pandas

## Import Kısmı

In [27]:
import pandas as pd
import re
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor

## Çalışma Ortamının Görüntülenmesi

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Çalışma Ortamı: {device.upper()}")

if device == "cuda":
    print(f"GPU Modeli: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("UYARI: CPU kullanıyorsunuz. Model çok yavaş çalışacaktır!")

Çalışma Ortamı: CUDA
GPU Modeli: NVIDIA GeForce RTX 4050 Laptop GPU
VRAM: 6.44 GB


## Model Kurulumu

In [3]:
model_name = "Qwen/Qwen2-VL-2B-Instruct"

processor = AutoProcessor.from_pretrained(model_name)

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="sdpa"
)

print("Model başarıyla yüklendi!")

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model başarıyla yüklendi!


## DATA

In [47]:
data = [
    {"food": "Tost", "meal_type": "Kahvaltı", "calories": 250},
    {"food": "Yulaf ezmesi", "meal_type": "Kahvaltı", "calories": 380},
    {"food": "Meyve salatası", "meal_type": "Kahvaltı", "calories": 90},
    {"food": "Simit", "meal_type": "Kahvaltı", "calories": 200},
    {"food": "Smoothie (meyve + yoğurt)", "meal_type": "Kahvaltı", "calories": 150},

    {"food": "Peynirli sandviç", "meal_type": "Öğle Yemeği", "calories": 300},
    {"food": "Fındık ve kuru meyve karışımı", "meal_type": "Öğle Yemeği", "calories": 500},
    {"food": "Pilav", "meal_type": "Öğle Yemeği", "calories": 500},
    {"food": "Izgara tavuk", "meal_type": "Öğle Yemeği", "calories": 400},
    {"food": "Sebze çorbası", "meal_type": "Öğle Yemeği", "calories": 150},

    {"food": "Izgara balık", "meal_type": "Akşam Yemeği", "calories": 350},
    {"food": "Sebzeli makarna", "meal_type": "Akşam Yemeği", "calories": 400},
    {"food": "Kinoa salatası", "meal_type": "Akşam Yemeği", "calories": 300},
    {"food": "Mercimek çorbası", "meal_type": "Akşam Yemeği", "calories": 180},
    {"food": "Fırında sebzeler", "meal_type": "Akşam Yemeği", "calories": 250},
]
df_food = pd.DataFrame(data)

## Prompt

In [48]:
question = "Bana kahvaltı için 1300 kalorilik bir menü öner. Yumurta yok."

meal_match = re.search(r"(kahvaltı|öğle yemeği|akşam yemeği)", question, re.IGNORECASE)
calorie_match = re.search(r"(\d+)\s*kalorilik", question, re.IGNORECASE)

meal_type = meal_match.group(1).capitalize() if meal_match else None
target_calories = int(calorie_match.group(1)) if calorie_match else None


## Menü Önerisi

In [51]:
def suggest_menu(df, meal_type, target_calories):
    df_meal = df[df["meal_type"] == meal_type]
    menu = []
    total = 0
    for _, row in df_meal.iterrows():
        if total + row["calories"] <= target_calories:
            menu.append(row["food"])
            total += row["calories"]
    return menu, total

menu, total = suggest_menu(df_food, meal_type, target_calories)

menu_text = f"Önerilen {meal_type} menüsü: {menu}, Toplam Kalori: {total}"

## Model Promptu

In [52]:
text_prompt = processor.apply_chat_template(
    [{"role": "user", "content": question + "\n" + menu_text}],
    tokenize=False,
    add_generation_prompt=True
)

## Model Inputunu

In [53]:
inputs = processor(
    text=[text_prompt],
    images=None,
    return_tensors="pt",
    padding=True
).to(model.device)

## Cevap Üretimi

In [54]:
output = model.generate(
    **inputs,
    max_new_tokens=256,
    do_sample=False,   
    temperature=0.0   
)

answer = processor.batch_decode(output, skip_special_tokens=True)[0]
response = answer.split("assistant")[-1].strip()

print("\n====== MODEL CEVABI ======\n")
print(response)


====== MODEL CEVABI ======

Önerilen Kahvaltı menüsü: ['Tost', 'Meyve salatası', 'Simit', 'Smoothie (meyve + yoğurt)']
Toplam Kalori: 1070


###### GİT Komutları

In [55]:
!git add .
!git commit -m "Notebook güncellendi"
!git push

[main 467f0be] Notebook gÃ¼ncellendi
 2 files changed, 326 insertions(+), 52 deletions(-)


To https://github.com/Naxyay/FET312_22040101051_GYMPT_7.git
   d576240..467f0be  main -> main
